In [1]:
import chess.pgn
import chess.engine
import chess.svg

import logging
logging.basicConfig(filename='./lichess.log')

from pprint import pprint
import matplotlib.pyplot as plt
import pickle as pkl

from utils import who, reduced_fen

## PARAMS
display_board = False
limit = chess.engine.Limit(time = 1) #time = 5
user = 'datashrimp'
move_limit = 5
logging_level = logging.INFO # logging.INFO # logging.WARNING
pgn_file = "./data/{}.pgn".format(user)
max_games = 100000
load = True
save = True

## RESET PGN READER

if load:
    counter = pkl.load(open('./store/{}/counter_{}.pkl'.format(user, move_limit), 'rb'))
    analysis = pkl.load(open('./store/{}/analysis_{}.pkl'.format(user, move_limit), 'rb'))
    fen_info = pkl.load(open('./store/fen_info_{}.pkl'.format(move_limit), 'rb'))
else:
    counter = 0
    analysis = {}
    fen_info = {}
    
pgn = open(pgn_file)
for i in range(counter):
    chess.pgn.skip_game(pgn)

logging.getLogger().setLevel(logging_level)



In [2]:
engine = chess.engine.SimpleEngine.popen_uci("./stockfish/Mac/stockfish-10-bmi2")

while counter < max_games:
    
    
    game = chess.pgn.read_game(pgn)
    board = game.board()
    
    result = game.headers['Result']
    new_pos_counter = 0
    
    if game.headers['White'] == user:
        user_colour = True
        if result == '1-0':
            user_result = 1
        if result == '0-1':
            user_result = 0
        logging.info("User is White")
    else:
        user_colour = False
        if result == '0-1':
            user_result = 1
        if result == '1-0':
            user_result = 0
        logging.info("User is Black")
        
    if result == '1/2-1/2':
        user_result = 0.5

    if result != '*':

        logging.info("Initial board")

        for move in game.mainline_moves():
            
            
            
            if board.fullmove_number <= move_limit:

                fen = reduced_fen(board.fen())
                san = board.san(move)

                logging.info("{}:{}:{}".format(board.fullmove_number, who(board.turn),  san))

                user_move = board.turn == user_colour

                if user_move:

                    if fen not in fen_info:
                        info = engine.analyse(board, limit) #, multipv = 4
                        fen_info[fen] = info
                    else:
                        info = fen_info[fen]

                    before_score = info["score"].pov(user_colour).score(mate_score=1000)
                    best_move = board.san(info['pv'][0])

                    logging.info("Best move:{}".format(best_move))
                    logging.info("Score before move:{}".format(before_score))

                if display_board:
                    display(chess.svg.board(board, flipped = not user_colour, lastmove = move, size = 400))

                board.push(move)

                if user_move:
                    new_fen = reduced_fen(board.fen())
                    if new_fen not in fen_info:
                        new_pos_counter += 1
                        
                        info = engine.analyse(board, limit) #, multipv = 4
                        fen_info[new_fen] = info
                    else:
                        info = fen_info[new_fen]

                    after_score = info["score"].pov(user_colour).score(mate_score=1000)
                    logging.info("Score after move:{}".format(after_score))
                    diff = after_score - before_score
                    logging.info("Score diff:{}".format(diff))

                    if (fen, san) not in analysis:
                        analysis[(fen, san)] = {'count': 0, 'colour': user_colour
                                                , 'move': san, 'fen': fen, 'diff': diff
                                                , 'best_move': best_move, 'before_score': before_score
                                               , 'after_score': after_score
                                               , 'result': 0
                                               , 'win_perc': 0}

                    analysis[(fen, san)]['count'] += 1
                    analysis[(fen, san)]['result'] += user_result
                    analysis[(fen, san)]['win_perc'] = (analysis[(fen, san)]['result']+0.0) / analysis[(fen, san)]['count']

                logging.info("")

            else:
    #             pprint(analysis)
                break
            
    counter += 1
    if save:
        with open('./store/{}/analysis_{}.pkl'.format(user, move_limit), 'wb') as file:
            pkl.dump(analysis, file)
        with open('./store/fen_info_{}.pkl'.format(move_limit), 'wb') as file:
            pkl.dump(fen_info, file)
        with open('./store/{}/counter_{}.pkl'.format(user, move_limit), 'wb') as file:
            pkl.dump(counter, file)
    
    print('{} new positions analysed'.format(new_pos_counter))
    print("{} games logged".format(counter))

4 new positions analysed
32 games logged
2 new positions analysed
33 games logged
4 new positions analysed
34 games logged
2 new positions analysed
35 games logged
5 new positions analysed
36 games logged
4 new positions analysed
37 games logged
3 new positions analysed
38 games logged
4 new positions analysed
39 games logged
4 new positions analysed
40 games logged
5 new positions analysed
41 games logged
4 new positions analysed
42 games logged
2 new positions analysed
43 games logged
3 new positions analysed
44 games logged
3 new positions analysed
45 games logged
4 new positions analysed
46 games logged
3 new positions analysed
47 games logged
3 new positions analysed
48 games logged
1 new positions analysed
49 games logged
2 new positions analysed
50 games logged
4 new positions analysed
51 games logged
3 new positions analysed
52 games logged
5 new positions analysed
53 games logged
1 new positions analysed
54 games logged
1 new positions analysed
55 games logged
3 new positions 

AttributeError: 'NoneType' object has no attribute 'board'

In [ ]:
analysis